## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-21-06-56-53 +0000,nypost,Ex-NFL sideline reporter Michele Tafoya files ...,https://nypost.com/2026/01/21/us-news/ex-nfl-s...
1,2026-01-21-06-46-00 +0000,nypost,Wife stabs estranged husband after learning he...,https://nypost.com/2026/01/21/world-news/uk-wi...
2,2026-01-21-06-42-52 +0000,bbc,Trump doubles down on Greenland ahead of Davos...,https://www.bbc.com/news/articles/cvgr19m642zo...
3,2026-01-21-06-39-33 +0000,bbc,'Really scary': Sydney residents react to shar...,https://www.bbc.com/news/videos/cpwnk040qd1o?a...
4,2026-01-21-06-33-47 +0000,nypost,Man who assassinated former Japanese prime min...,https://nypost.com/2026/01/21/world-news/japan...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2361/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
25,trump,99
27,greenland,42
379,new,18
207,court,16
29,davos,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
350,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...,203
75,2026-01-21-01-05-00 +0000,wsj,President Trump’s campaign for Greenland follo...,https://www.wsj.com/politics/policy/trumps-gre...,187
34,2026-01-21-03-26-41 +0000,nypost,Trump teases how far he’ll go to acquire Green...,https://nypost.com/2026/01/20/us-news/trump-te...,185
156,2026-01-20-22-10-25 +0000,wapo,Stocks slide after Trump threatens new tariffs...,https://www.washingtonpost.com/business/2026/0...,182
353,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...,179


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
350,203,2026-01-20-10-36-28 +0000,bbc,"Greenland 'will stay Greenland', former Trump ...",https://www.bbc.com/news/articles/c98p6m0r53no...
353,69,2026-01-20-10-30-00 +0000,wsj,Wednesday’s Supreme Court arguments over wheth...,https://www.wsj.com/economy/central-banking/je...
328,64,2026-01-20-12-09-26 +0000,cbc,EU chief promises 'unflinching' response to ne...,https://www.cbc.ca/news/world/greenland-europe...
70,60,2026-01-21-01-17-00 +0000,wsj,Escalating tensions over Greenland are superch...,https://www.wsj.com/economy/global/united-stat...
291,60,2026-01-20-15-58-06 +0000,nypost,Trump tells The Post why he posted Emmanuel Ma...,https://nypost.com/2026/01/20/us-news/trump-re...
23,52,2026-01-21-04-15-28 +0000,nypost,Air Force One forced to turn around after Trum...,https://nypost.com/2026/01/20/us-news/air-forc...
154,49,2026-01-20-22-12-28 +0000,nyt,Minnesota Police Leaders Criticize Federal Tac...,https://www.nytimes.com/2026/01/20/us/ice-minn...
120,47,2026-01-20-23-23-02 +0000,nyt,Photographer Discovers Train Part That Could H...,https://www.nytimes.com/2026/01/20/world/europ...
299,44,2026-01-20-15-15-07 +0000,latimes,Netflix amends Warner Bros. deal to all cash i...,https://www.latimes.com/entertainment-arts/bus...
252,41,2026-01-20-18-03-00 +0000,wsj,A group aligned with Senate Majority Leader Jo...,https://www.wsj.com/politics/policy/gop-spendi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
